In [77]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('../food_inflation_analysis.db')

# Get a list of all tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Create an empty DataFrame to store the combined data
combined_data = pd.DataFrame()



In [78]:
# Create a dictionary to store the DataFrames
dfs = {}

# Iterate over each table
for table in tables:
    table_name = table[0]
    
    # Read the table data into a DataFrame
    dfs[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)




In [79]:
tables

[('OCED_USA_FOOD_INFLATION',),
 ('DOW_JONES_REAL',),
 ('USA_MEAT_EXPORT_IMPORT',),
 ('food_production',),
 ('interest_rate',),
 ('gas_prices',)]

In [64]:
merged_df = pd.merge(dfs['gas_prices'], dfs['OCED_USA_FOOD_INFLATION'], on='Date', how='inner')

In [65]:
merged_df.rename({'Price':'gas_price'}, axis=1,inplace=True)

In [66]:
merged_df

,Date,gas_price,Inflation
0,1990-09-01 00:00:00,1.258,6.320000
1,1990-10-01 00:00:00,1.335,6.379585
2,1990-11-01 00:00:00,1.324,6.359301
3,1990-12-01 00:00:00,1.251,5.770751
4,1991-01-01 00:00:00,1.153,4.122138
...,...,...,...
392,2023-05-01 00:00:00,3.555,5.647190
393,2023-06-01 00:00:00,3.571,4.581234
394,2023-07-01 00:00:00,3.597,3.485868
395,2023-08-01 00:00:00,3.840,2.893452


In [67]:
merged_df = pd.merge(merged_df, dfs['DOW_JONES_REAL'], on='Date', how='inner')
merged_df

,Date,gas_price,Inflation,real-price
0,1990-09-01 00:00:00,1.258,6.320000,5687.30
1,1990-10-01 00:00:00,1.335,6.379585,5632.01
2,1990-11-01 00:00:00,1.324,6.359301,5887.20
3,1990-12-01 00:00:00,1.251,5.770751,6057.42
4,1991-01-01 00:00:00,1.153,4.122138,6258.12
...,...,...,...,...
392,2023-05-01 00:00:00,3.555,5.647190,33303.17
393,2023-06-01 00:00:00,3.571,4.581234,34717.27
394,2023-07-01 00:00:00,3.597,3.485868,35808.45
395,2023-08-01 00:00:00,3.840,2.893452,34791.35


In [68]:
merged_df.rename({'real-price':'dow_jones_real-price'}, axis=1,inplace=True)

In [69]:
merged_df

,Date,gas_price,Inflation,dow_jones_real-price
0,1990-09-01 00:00:00,1.258,6.320000,5687.30
1,1990-10-01 00:00:00,1.335,6.379585,5632.01
2,1990-11-01 00:00:00,1.324,6.359301,5887.20
3,1990-12-01 00:00:00,1.251,5.770751,6057.42
4,1991-01-01 00:00:00,1.153,4.122138,6258.12
...,...,...,...,...
392,2023-05-01 00:00:00,3.555,5.647190,33303.17
393,2023-06-01 00:00:00,3.571,4.581234,34717.27
394,2023-07-01 00:00:00,3.597,3.485868,35808.45
395,2023-08-01 00:00:00,3.840,2.893452,34791.35


In [70]:
# 检查 'Date' 列的数据类型
print(merged_df['Date'].dtype)

# 如果 'Date' 列的数据类型不是 datetime，将其转换为 datetime
if merged_df['Date'].dtype != 'datetime64[ns]':
    merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# 从 'Date' 列中提取年份并将其存储在新的 'Year' 列中
merged_df['Year'] = merged_df['Date'].dt.year

object


In [71]:
merged_df

,Date,gas_price,Inflation,dow_jones_real-price,Year
0,1990-09-01,1.258,6.320000,5687.30,1990
1,1990-10-01,1.335,6.379585,5632.01,1990
2,1990-11-01,1.324,6.359301,5887.20,1990
3,1990-12-01,1.251,5.770751,6057.42,1990
4,1991-01-01,1.153,4.122138,6258.12,1991
...,...,...,...,...,...
392,2023-05-01,3.555,5.647190,33303.17,2023
393,2023-06-01,3.571,4.581234,34717.27,2023
394,2023-07-01,3.597,3.485868,35808.45,2023
395,2023-08-01,3.840,2.893452,34791.35,2023


In [72]:
# 将dfs['USA_MEAT_EXPORT_IMPORT']和merge_df按照Year列进行合并
merge_df = pd.merge(merged_df, dfs['USA_MEAT_EXPORT_IMPORT'], on='Year', how='inner')

merge_df


,Date,gas_price,Inflation,dow_jones_real-price,Year,Exports,Imports
0,1990-09-01,1.258,6.320000,5687.30,1990,1.457129e+10,1.784900e+10
1,1990-10-01,1.335,6.379585,5632.01,1990,1.457129e+10,1.784900e+10
2,1990-11-01,1.324,6.359301,5887.20,1990,1.457129e+10,1.784900e+10
3,1990-12-01,1.251,5.770751,6057.42,1990,1.457129e+10,1.784900e+10
4,1991-01-01,1.153,4.122138,6258.12,1991,1.712753e+10,1.741726e+10
...,...,...,...,...,...,...,...
392,2023-05-01,3.555,5.647190,33303.17,2023,7.199305e+10,1.524754e+10
393,2023-06-01,3.571,4.581234,34717.27,2023,7.199305e+10,1.524754e+10
394,2023-07-01,3.597,3.485868,35808.45,2023,7.199305e+10,1.524754e+10
395,2023-08-01,3.840,2.893452,34791.35,2023,7.199305e+10,1.524754e+10


In [73]:
merged_df = pd.merge(merged_df, dfs['Cereal_Production'], on='Year', how='inner')
merged_df

,Date,gas_price,Inflation,dow_jones_real-price,Year,Cereal_Production,Food_Production,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9
0,1990-09-01,1.258,6.320000,5687.30,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89
1,1990-10-01,1.335,6.379585,5632.01,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89
2,1990-11-01,1.324,6.359301,5887.20,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89
3,1990-12-01,1.251,5.770751,6057.42,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89
4,1991-01-01,1.153,4.122138,6258.12,1991,280063390.6,67.32,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2021-08-01,3.158,2.980927,39780.82,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90
372,2021-09-01,3.175,4.516333,37972.88,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90
373,2021-10-01,3.291,5.382061,39867.17,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90
374,2021-11-01,3.395,6.385879,38173.48,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90


In [74]:
# 检查 'Date' 列的数据类型
print(dfs['interest_rate']['Date'].dtype)

# 如果 'Date' 列的数据类型不是 datetime，将其转换为 datetime
if dfs['interest_rate']['Date'].dtype != 'datetime64[ns]':
    dfs['interest_rate']['Date'] = pd.to_datetime(dfs['interest_rate']['Date'])

object


In [75]:
merged_df = pd.merge(merged_df, dfs['interest_rate'], on='Date', how='inner')
merged_df

,Date,gas_price,Inflation,dow_jones_real-price,Year,Cereal_Production,Food_Production,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,FEDFUNDS
0,1990-09-01,1.258,6.320000,5687.30,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89,8.20
1,1990-10-01,1.335,6.379585,5632.01,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89,8.11
2,1990-11-01,1.324,6.359301,5887.20,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89,7.81
3,1990-12-01,1.251,5.770751,6057.42,1990,312410604.0,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,65.89,7.31
4,1991-01-01,1.153,4.122138,6258.12,1991,280063390.6,67.32,68.81,66.52,60.46,65.48,66.19,68.59,65.41,57.78,66.70,6.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2021-08-01,3.158,2.980927,39780.82,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90,0.09
372,2021-09-01,3.175,4.516333,37972.88,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90,0.08
373,2021-10-01,3.291,5.382061,39867.17,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90,0.08
374,2021-11-01,3.395,6.385879,38173.48,2021,452628438.9,105.46,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90,0.08


In [76]:
# Close the database connection
conn.close()